In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)


In [ ]:
pip install scanpy

In [ ]:
adata = sc.read_h5ad("healthy_hamstring_tendon.h5ad")

In [ ]:
adata

In [ ]:
adata_raw = adata.raw.to_adata()

In [ ]:
import os

# Change to a directory you have access to, like your home directory or a relative path
results_dir = "./healthy_hamstring/write/final_versions"

# Create the directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Now write the file
results_file = os.path.join(results_dir, "healthy_hamstring_processed_adata_raw_nonormalization.h5ad")
adata_raw.write(results_file)


In [ ]:
adata_raw

In [ ]:
# Filtering low quality cells

# mitochondrial genes
adata_raw.var["mt"] = adata_raw.var_names.str.startswith("mt-")
# ribosomal genes
adata_raw.var["ribo"] = adata_raw.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata_raw.var["hb"] = adata_raw.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata_raw, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata_raw

In [ ]:
p1 = sns.displot(adata_raw.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata_raw, "tissue")
p3 = sc.pl.scatter(adata_raw, "total_counts", "n_genes_by_counts", color="mt")

In [ ]:
adata_raw.var_names_make_unique()

In [ ]:
sc.pl.highest_expr_genes(adata_raw, n_top=20)

In [ ]:
print("Min:", adata_raw.X.min(), "Max:", adata_raw.X.max(), "Mean:", adata_raw.X.mean())

In [ ]:
sc.pp.filter_cells(adata_raw, min_genes=200)
sc.pp.filter_genes(adata_raw, min_cells=3)

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

# Generate scatter plot and return axis object
ax = sc.pl.scatter(adata_raw, x="total_counts", y="n_genes_by_counts", show=False)

# Add a red horizontal line at y = 2500
ax.axhline(y=2500, color='red', linestyle='--', linewidth=1.5)

# Show the modified plot
plt.show()



In [ ]:
adata_raw = adata_raw[adata_raw.obs.n_genes_by_counts < 2500, :]
adata_raw = adata_raw[adata_raw.obs.pct_counts_mt < 10, :].copy()

In [ ]:
adata_raw

In [ ]:
adata_raw.write(results_file)

In [ ]:
adata_normalized.write(results_file)

In [ ]:
import os

# Change to a directory you have access to, like your home directory or a relative path
results_dir = "./healthy_hamstring/write/final_versions/normalized_data"

# Create the directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Now write the file
results_file = os.path.join(results_dir, "healthy_hamstring_processed_adata_raw_normalized.h5ad")
adata.write(results_file)

In [ ]:
adata_normalized = adata_raw.copy()

In [ ]:
sc.pp.normalize_total(adata_normalized, target_sum=1e4)
sc.pp.log1p(adata_normalized)

In [ ]:
print("Min:", adata_normalized.X.min(), "Max:", adata_normalized.X.max(), "Mean:", adata_normalized.X.mean())

In [ ]:
sc.pp.filter_cells(adata_raw, min_genes=200)
sc.pp.filter_genes(adata_raw, min_cells=3)
adata_raw = adata_raw[adata_raw.obs.n_genes_by_counts < 2500, :]
adata_raw = adata_raw[adata_raw.obs.pct_counts_mt < 10, :].copy()

In [ ]:
#save adata_check_raw
sc.pp.normalize_total(adata_check_raw)
sc.pp.log1p(adata_check_raw)
#save norm

In [ ]:
print("Min:", adata.X.min(), "Max:", adata.X.max(), "Mean:", adata.X.mean())

In [ ]:
print("Min:", adata_check_raw.X.min(), "Max:", adata_check_raw.X.max(), "Mean:", adata_check_raw.X.mean())

In [ ]:
adata2 = sc.read("healthy_hamstring/write/healthy_hamstring_processed_test2.h5ad")

In [ ]:
adata2

In [ ]:
adata.obs["donor_id"]

In [ ]:
adata.obs["mitoRatio"]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 5))
for i, c in enumerate(["cell_type", "disease"]):
    sc.pl.umap(adata, color=[c], ax=axs[i], show=False)
    axs[i].set_axis_off()
plt.tight_layout()
plt.show()